In [1]:
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt



import random

# Queuing-as df first
To Do:  
* run better variables for timing customer incident rates
* more interesting call times
* factor in timed breaks time for reps
* factor in random breaks for reps
* bring in new reps for a timeframe
* let reps leave for the days

Later: 
* Abstract this to language types
  
Later: transaction logs as classes with add node function?

In [2]:
[int(sp.random.lognormal(mean=5, sigma=1))]

[56]

In [ ]:
call_time_dist

In [14]:
num_customerz = 10*24
time_periodz = 24*60
num_repz = 50

In [15]:
range(20)

range(0, 20)

In [16]:
def generate_customer_table(num_customers, time_periods, seed=None):
    if seed is not None:
        np.random.seed(seed)

    df_customers = pd.DataFrame([i for i in range(1,num_customers+1)], columns=['customer_id'])
    df_customers['required_call_time'] = [np.random.choice([1,2,3,4,5]) for i in range(num_customers)]
    df_customers['time_in']=[np.random.choice(range(1,time_periods)) for i in range(num_customers)]
    df_customers['time_out']=[0 for i in range(num_customers)]
    df_customers['time_assigned'] = [0 for i in range(num_customers)]
    df_customers['assigned_rep'] = ['none' for i in range(num_customers)]

    df_customers=df_customers.sort_values(by='time_in')
    df_customers['customer_id'] = [i for i in range(1,num_customers+1)]
    
    return df_customers
# df_customers

def generate_qs(time_periods):

    df_q = pd.DataFrame([0 for x in range(time_periods*2)], columns=['customer_q_list'])
    df_q['customer_q_list'] = [[] for x in range(time_periods*2)]
    df_q['update_event'] = [0 for x in range(time_periods*2)]

    return df_q

def generate_reps(num_reps, time_periods):
    reps_list = ['rep' + str(i) + '_busy' for i in range(1,num_reps+1)]
    df_reps = pd.DataFrame(columns=reps_list)
    for rep in reps_list:
        df_reps[rep] = list(range(time_periods*2))
#     df_reps['rep1_busy']=[0 for x in range(time_periods*2)]
#     df_reps['rep2_busy']=[0 for x in range(time_periods*2)]
#     df_reps['rep3_busy']=[0 for x in range(time_periods*2)]
    #df_assigned = pd.DataFrame(columns=['rep1_busy_with', 'rep2_busy_with'], index=df_reps.index)
    
    #Next: Scheduled and random breaks
    
    return df_reps

In [17]:
def run_day(df_customers, df_q, df_reps, iteration, time_periods):
    for t in range(time_periods*2):

        df_q.at[t, 'customer_q_list'] = list(df_customers[(df_customers.time_assigned==0)&(df_customers.time_in<=t)].customer_id)


        if len(df_q['customer_q_list'][t])!=0:

            for customer_qued in df_q['customer_q_list'][t]:

                for rep in df_reps.columns:
                    if df_reps[rep][t]==0:
                        _req_call_time = int(df_customers[df_customers.customer_id==customer_qued]['required_call_time'])
                        df_customers.at[df_customers.customer_id==customer_qued, ['time_assigned', 'time_out',  'assigned_rep']] = [t, t+_req_call_time , rep]

                        df_reps.loc[t: t + _req_call_time, rep] = 1
                        #df_assigned.loc[t: t + _req_call_time , rep+'_with'] = customer_qued
                        break
        df_customers['wait_time'] = df_customers.time_assigned - df_customers.time_in
        df_customers['iteration'] = iteration
        
        df_txn_log = pd.concat([df_q, df_reps],axis=1)
        df_txn_log['num_in_q'] = df_txn_log.customer_q_list.apply(lambda x: len(x))
        df_txn_log['iteration'] = iteration
        df_txn_log = df_txn_log.iloc[:df_customers.time_out.max()+1, :]
    return df_customers, df_txn_log

In [19]:
%%time
df_customers, df_txn_log = run_day(generate_customer_table(num_customers=num_customerz, time_periods=time_periodz, seed=42), 
                                   generate_qs(time_periodz), 
                                   generate_reps(num_repz, time_periodz), 
                                   iteration=1, 
                                  time_periods=time_periodz)

"""
Timed: 5 min 30 s
num_customerz = 10*24
time_periodz = 24*60
num_repz = 50
"""

Wall time: 5min 30s


In [10]:
for i in range(2,10):
    df_a, df_b = run_day(generate_customer_table(), 
                                   generate_qs(), 
                                   generate_reps(), 
                                   iteration=i)
    df_customers = pd.concat([df_customers, df_a])
    df_txn_log = pd.concat([df_txn_log, df_b])

NameError: name 'time_periods' is not defined

In [32]:
# Wait Times
np.quantile(df_customers.wait_time, [.25, .5, .99])

array([ 8.  , 18.  , 41.01])

In [33]:
# Rep utilization rates
df_txn_log.loc[:, ['rep1_busy', 'rep2_busy', 'rep3_busy']].mean()


rep1_busy    0.977866
rep2_busy    0.969170
rep3_busy    0.938340
dtype: float64